#Limpieza manual

En este cuadernillo se agrupa el proceso de los siguientes pasos:

*   Transformación a intervalos de atributos
*   Aplicación de RDP
*   Limpieza manual

La aplicación de rdp se reduce al método "rdp_compute_mean".



In [ ]:
pip install rdp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rdp: filename=rdp-0.8-py3-none-any.whl size=4588 sha256=341340491b61305067072c73533d37a165b1746ba0ffc195787eb8eb2f4c6710
  Stored in directory: /root/.cache/pip/wheels/ee/30/35/b52f1e297bfd39e8a51f203dc46dc50cf411a6cc151296b13f
Successfully built rdp


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.svm import LinearSVC
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import sys

np.set_printoptions(threshold=sys.maxsize)

In [ ]:
#IMPORTS

import pandas as pd
import numpy as np
import math 
from os import listdir
from rdp import rdp

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
#PATHS

path_sin=r"/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_sin_RDP/"
path_con=r"/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP/"
outputPath = r"/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/"

In [ ]:
#MÉTODOS UTILIZADOS

#Removes x,y,z points that are out of a stablished range from dataframe 
#PARAMETERS: dataframe
#RETURNS the dataframe with intervals

def compute_intervals(df):
  new_data=pd.DataFrame()
  for i in range(len(df)-1):
    for column in df.columns.drop(['drone_model']):
      if i<len(df)-1 and column!='drone_model':
        new_data.loc[i, column] = df.loc[i+1,column]-df.loc[i,column]
  new_data['drone_model']=df['drone_model']
  return new_data

#Gets the name of a drone model given its code
#PARAMETERS: float
#RETURNS string with the name of the drone model

def get_name(f):
  list_class = [1030.0, 2100.0, 12001.0, 10016.0, 6001.0]
  list_names= ['Fixed_wing','Plane','Octorotor','Quadrotor','Hexarotor']
  return list_names[list_class.index(f)]


#COMPUTES THE MEAN OF THE ATTRIBUTES BETWEEN THE RDP SELECTED POINTS to a dataframe (list of 3d points)
#PARAMETERS: dataframe to transform and epsilon (umbral de distancia)
#RETURNS the reduced dataframe with all atributtes

def rdp_compute_mean(df,e):

  np_data=df[["x","y","z"]].to_numpy()

  rdp_mask = rdp(np_data, epsilon=e,return_mask=True)

  rdp_df=df[rdp_mask]

  mean_df=pd.DataFrame(index=range(len(rdp_df.index)))

  j=0
  mean_counter=0

  for i in range(len(df)):
    mean_df = mean_df.fillna(0)

    if i==0 and rdp_mask[i]:
      for col in df.columns.drop(['timestamp','drone_model','x','y','z']):
        mean_df.loc[j,col]=df.loc[i,col]
      mean_counter=1 

    if rdp_mask[i] and i!=0 and j<len(rdp_df):
      mean_counter+=1
      for col in df.columns.drop(['timestamp','drone_model','x','y','z']):
        mean_df.loc[j,col]+=df.loc[i,col]
        mean_df.loc[j,col]=mean_df.loc[j,col]/mean_counter
      mean_counter=1
      j+=1
      for col in df.columns.drop(['timestamp','drone_model','x','y','z']):
        mean_df.loc[j,col]=df.loc[i,col]

    if not(rdp_mask[i]) and j<len(rdp_df):
      for col in df.columns.drop(['timestamp','drone_model','x','y','z']):
        mean_df.loc[j,col]+=df.loc[i,col]
      mean_counter+=1 
    mean_df = mean_df.fillna(0)
      
  rdp_df = rdp_df.reset_index(drop=True)
  mean_df[['x','y','z','timestamp','drone_model']]=rdp_df[['x','y','z','timestamp','drone_model']]
  mean_df = mean_df[['timestamp','x','y','z','vx','vy','vz','ax','ay','az','drone_model']]

  return mean_df

In [ ]:
#MAIN CODE: APPLIES RDP AND SAVES ALL RESULTS IN A DATAFRAME (new_df2)
path=r"/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_sin_RDP/"
e = 0.01

new_df2= pd.DataFrame()

listas=['Quadrotor','Octorotor','Hexarotor','Plane','Fixed_wing']

for dir in listdir(path):
  if dir in listas:
    for f in listdir(path+dir): 
        print(dir+'/'+f)
        test = pd.DataFrame()
        df = pd.DataFrame()
        df = pd.read_csv(path+dir+'/'+f,sep=',')

        print("points before rdp: "+str(df.shape))
        if len(df)>300 and (dir=='Fixed_wing' or dir=='Plane' or dir=='Quadrotor'):# applies rdp to 3 drone models
          test=rdp_compute_mean(df,e)
        else:
          test = df.copy()

        print("points after rdp: "+str(test.shape))

        #test.to_csv(outputPath+dir+'/'+f,index=False) #this line saves the new csv with rdp in the output path

        new_data = pd.DataFrame()
        new_data = compute_intervals(df)

        print('------------------------')

        new_df2=pd.concat([new_df2,new_data],ignore_index=True)

new_df2

Quadrotor/004199e3-9cfa-4f1e-93c9-175f7e92e514.csv
points before rdp: (2560, 11)
points after rdp: (2051, 11)
Instance shape: (2559, 11)
------------------------
Quadrotor/00f9ba00-1ceb-4657-8b0b-7b5140da6b0f.csv
points before rdp: (1518, 11)
points after rdp: (1084, 11)
Instance shape: (1517, 11)
------------------------
Quadrotor/01f043e0-8317-49c8-badb-425f6dd94fa0.csv
points before rdp: (1409, 11)
points after rdp: (815, 11)
Instance shape: (1408, 11)
------------------------
Quadrotor/02bbabd0-a34e-45f1-ae59-e0dca358ab86.csv
points before rdp: (4068, 11)
points after rdp: (1857, 11)
Instance shape: (4067, 11)
------------------------
Quadrotor/0037bf03-a798-4207-b01a-605315c17490.csv
points before rdp: (2215, 11)
points after rdp: (1013, 11)
Instance shape: (2214, 11)
------------------------
Quadrotor/0039e8c9-883d-48a2-aa7b-b80545e45a61.csv
points before rdp: (2685, 11)
points after rdp: (1425, 11)
Instance shape: (2684, 11)
------------------------
Quadrotor/01c15f92-c8c9-4276-

,timestamp,x,y,z,vx,vy,vz,ax,ay,az,drone_model
0,4000.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10016.0
1,92000.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10016.0
2,104000.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10016.0
3,100000.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10016.0
4,96000.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10016.0
...,...,...,...,...,...,...,...,...,...,...,...
264173,109881.0,-0.010410,-0.004805,0.022227,-0.000505,-0.001247,0.001717,-0.089650,-0.057040,-0.010264,12001.0
264174,89903.0,-0.007646,-0.003738,0.018988,0.000696,0.000764,0.001944,0.006012,-0.027889,0.068936,12001.0
264175,109881.0,-0.009096,-0.004777,0.022149,0.002046,-0.001319,0.000200,0.087672,-0.018727,-0.203077,12001.0
264176,99900.0,-0.007593,-0.004457,0.020534,0.000720,0.001930,0.001936,-0.055466,0.195220,0.016572,12001.0


In [ ]:
#To show 50 trajectory 3D graphs of the same folder

for dir in listdir(outputPath):
  counter=0
  
  if dir!='Output_test' and dir!='bad' and dir!='VTOL' and dir=='Fixed_wing':
    for f in listdir(outputPath+dir): 
        fig = plt.figure(figsize=(6,120))
        
        df1 = pd.read_csv(outputPath+dir+'/'+f,sep=',')
        print(f)

        x = df1['x']
        y = df1['y']
        z = df1['z']

        counter+=1

        if counter==51:
          break

        ax1 = fig.add_subplot(50,1,counter, projection='3d')
        ax1.scatter(x, y, z,
                  linewidths=1, alpha=.7,
                  edgecolor='k',
                  s = 200,
                  c=z)
        plt.tight_layout()
        plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#Limpieza manual, lee cada archivo y borra outliers vistos

i = '032877d8-4de9-4c0b-95c9-106c0813d480'
folder='Quadrotor'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'z'] >-2]
df2=df2[df2.loc[:,'x'] <0.1]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   

i = '03df3989-d395-44c7-beef-89574057863c'
folder='Quadrotor'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'z'] >-10]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   

i = '015b7d4a-36ce-4196-9384-17882c5cee64'
folder='Quadrotor'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'z'] >0]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '0d761381-56b2-49b7-acfe-5ca51c43ebe2'
folder='Quadrotor'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'z'] <0.6]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   



i = '06e6e5c0-ed66-42b7-828e-ee9c2da5025a'
folder='Quadrotor'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'y'] <-1]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '0568034e-6c34-46f1-a0bc-f5a810596cf1'
folder='Quadrotor'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'z'] >-2]
df2=df2[df2.loc[:,'x'] <0.1]
df2=df2[df2.loc[:,'y'] >-0.15]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '141a8fe1-3b7d-4ee2-a93b-e1f1f2cbb47b'
folder='Hexarotor'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'x'] >-0.06]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '32b1b5d9-4c4b-4c8c-ab7b-a9df3a6ca557'
folder='Hexarotor'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'x'] >-0.06]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '4c2e93e1-e7aa-41b6-9eeb-42fd3229f0cb'
folder='Hexarotor'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'z'] >-2.5]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '505320f9-ffe8-4a91-ab81-907045d7b9b0'
folder='Hexarotor'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'y'] >1]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '55483998-ee2e-43a0-8c59-7415ca105e4f'
folder='Hexarotor'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2.iloc[:-58]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = 'dd1291aa-5f06-4b3f-909b-f15d0706a706'
folder='Octorotor'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'z'] >-2]
df2=df2[df2.loc[:,'x'] >0.5]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '93c1fa01-5879-42a9-a9d1-6f3c768ae989'
folder='Octorotor'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'y'] >0]
df2=df2[df2.loc[:,'x'] <11]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '18067bdf-7d7c-4dd0-867d-51483a30a389'
folder='Plane'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'z'] <70]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   



i = '20981598-82ae-43f1-bd70-ca58ca875113'
folder='Plane'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'z'] >-0.25]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '5b3e734f-8be3-4c5d-9324-31009fe809ea'
folder='Plane'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'x'] >-55]
df2=df2.iloc[:-265]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '9010ca00-f910-47ee-8199-9c34ce763afd'
folder='Plane'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2.iloc[:-536]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '83147ffb-01ce-44dc-8b88-6741832aed62'
folder='Plane'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2.iloc[:-536]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '34c63d83-f917-4a04-a8f1-63615cfd084e'
folder='Fixed_wing'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'z'] >-100]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '56cdb445-5aa7-428a-a6ec-5159cc4bb6d4'
folder='Fixed_wing'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2.iloc[:-80]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '678b79c7-a90e-4bd7-9c9c-aa8bafdc6167'
folder='Fixed_wing'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'z'] >0.004]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '6c42c828-3c41-4626-b005-5845813da18f'
folder='Fixed_wing'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'z'] <0.02]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '723b81f1-87c9-48c2-bb87-e537fdb5729a'
folder='Fixed_wing'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'z'] >-400]
df2=df2[df2.loc[:,'z'] <-0.05]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '8617fe45-3269-4098-9165-e94b176d74a4'
folder='Fixed_wing'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'y'] <20000]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '9db1ed81-187b-4cf5-9f80-c9e47dc4888c'
folder='Fixed_wing'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2.iloc[:-600]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '9e370116-2353-4227-9d36-6b6e17dd0eed'
folder='Fixed_wing'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2[df2.loc[:,'x'] <0]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = 'fc830012-c75c-46e0-bc3a-30b10f89ca97'
folder='Fixed_wing'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2.loc[df2["z"] < -12]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = '51088318-7490-405b-9add-24dc2a6388ce'
folder='Fixed_wing'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2.loc[df2["z"] > -400]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = 'c65fa641-6b8b-4bab-8921-283492d97db1'
folder='Fixed_wing'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2.loc[df2["x"] > 10]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)   


i = 'fed0f339-4d9b-4f0f-bef1-334b22eb1d4d'
folder='Fixed_wing'
df1 = pd.read_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',sep=',')
df2=df1.copy()
df2=df2.loc[df2["z"] < 0.12]
df2.to_csv('/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/'+folder+'/'+i+'.csv',index=False)  

In [ ]:
#Código base para guardar datos con limpieza en un csv y testear (sin aplicar rdp)

path_sin=r"/content/drive/MyDrive/TFG Celia/Código/Datos_limpios/Datos_con_RDP_limpieza_manual/"

new_df= pd.DataFrame()

listas=['Quadrotor','Octorotor','Hexarotor','Plane','Fixed_wing']

for dir in listdir(path_sin):
  if dir in listas:
    for f in listdir(path_sin+dir): 
        print(dir+'/'+f)
        test = pd.DataFrame()
        df = pd.DataFrame()
        df = pd.read_csv(path_sin+dir+'/'+f,sep=',')

        #test.to_csv(outputPath+dir+'/'+f,index=False)  #this line saves the new csv with rdp in the output path

        new_data = pd.DataFrame()
        print("Before: "+str(df.shape))

        new_data = compute_intervals(df)

        print(new_data.shape)
        print("After: "+str(new_data.shape))
        print('------------------------')

        new_df=pd.concat([new_df,new_data],ignore_index=True)

new_df.to_csv('/content/drive/MyDrive/TFG Celia/Código/Resultados/test3.csv',index=False)  

Plane/022eb50b-d1eb-44a7-9b2b-04d54e359e42.csv
Before: (114, 11)
(113, 11)
After: (113, 11)
------------------------
Plane/0b3c8bfb-142e-4903-ab6c-1e4070a21eac.csv
Before: (1925, 11)
(1924, 11)
After: (1924, 11)
------------------------
Plane/0e19ad39-ed6d-4680-95b1-e74184f7cf74.csv
Before: (831, 11)
(830, 11)
After: (830, 11)
------------------------
Plane/1433cb06-9a99-4520-9fe9-fe16444fddaf.csv
Before: (1099, 11)
(1098, 11)
After: (1098, 11)
------------------------
Plane/1b75e769-4ee1-4edd-86d6-e8f4f8d42fa9.csv
Before: (665, 11)
(664, 11)
After: (664, 11)
------------------------
Plane/236442d9-e3b9-444c-918e-07f1fb58d99a.csv
Before: (72, 11)
(71, 11)
After: (71, 11)
------------------------
Plane/2769929c-937a-4ce2-9dc1-e91fa06b6a06.csv
Before: (144, 11)
(143, 11)
After: (143, 11)
------------------------
Plane/29a4ea41-bc2b-4835-a6fd-43719a78baa4.csv
Before: (1834, 11)
(1833, 11)
After: (1833, 11)
------------------------
Plane/2cafd114-9e9d-4a86-bf61-e258f76880aa.csv
Before: (16

In [ ]:
#MÉTODOS NO UTILIZADOS/ÚTILES

def power(x,x2):
  result=(x-x2)**2
  return result

def sed(x,y,z):
  result=math.sqrt(x+y+z)
  return result

#Applies Douglas Peucker to a dataframe (list of 3d points)
#PARAMETERS: dataframe to transform and epsilon (umbral de distancia)
#RETURNS the reduced dataframe test

def apply_rdp(df,e):

  np_data=df[["x","y","z"]].to_numpy()
  red_data = rdp(np_data, epsilon=e)
  test = pd.DataFrame(red_data)
  test.columns = ['x', 'y', 'z']

  return test

def remove_timestamp_outliers(df):

  q75,q25 = np.percentile(df.loc[:,'timestamp'],[75,25])
  in_qr = q75-q25
 
  max = q75+(1.5*in_qr)
  min = q25-(1.5*in_qr)

  df=df[df.loc[:,'timestamp'] < max]
  df=df[df.loc[:,'timestamp'] > min]

  return df
  
def is_linear(df):
  count=0
  for column in df.columns.drop(['timestamp','drone_model','vx','vy','vz','ax','ay','az']):
    if df[column].mean() == 0:
      count+=1
      if count==2: 
        print('df is linear')
        return True
      else:
        return False

def remove_outliers(df):
  dim_counter=0

  for column in df.columns:
    q75,q25 = np.percentile(df.loc[:,column],[75,25])
    if column == 'x':
      q75,q25 = np.percentile(df.loc[:,column],[75,25])
      in_qr = q75-q25
 
      ymax = q75+(1.5*in_qr)
      ymin = q25-(1.5*in_qr)

      dim_counter +=1
    if column == 'y':
      q75,q25 = np.percentile(df.loc[:,column],[75,25])
      in_qr = q75-q25
 
      xmax = q75+(1.5*in_qr)
      xmin = q25-(1.5*in_qr)

      dim_counter +=1
    if column == 'z':
      q75,q25 = np.percentile(df.loc[:,column],[75,25])
      in_qr = q75-q25
      zmax = q75+(1.5*in_qr)
      zmin = q25-(1.5*in_qr)

      dim_counter +=1
    
    if dim_counter is 3:
      #df2 = df[(df.loc[:,'x'] > xmin) & (df.loc[:,'y'] > ymin) & (df.loc[:,'z'] > zmin)]
      #df2 = df[(df.loc[:,'x'] < xmax) & (df.loc[:,'y'] < ymax) & (df.loc[:,'z'] < zmax)]
      df2 = df[df.loc[:,'x'] < xmax]
      df2 = df2[df.loc[:,'y'] < ymax]
      df2 = df2[df.loc[:,'z'] < zmax]
      df2 = df2[df.loc[:,'x'] > xmin]
      df2 = df2[df.loc[:,'y'] > ymin]
      df2 = df2[df.loc[:,'z'] > zmin]

  return df2

def calculate_epsilon(df):
  e=(len(df)/(3*(0.75*len(df))))*0.2
  #epsilon := (totalNumberOfPoints / (3 * desiredNumberOfPoints)) * 2
  return e